# FACS Analysis

This Jupyter notebook walks you through the basic steps of analyzing Mass-Titr FACS results. For high-throughput binding experiments, the `highThroughputScripts` pipeline is available on [GitHub](https://github.com/KeatingLab/highThroughputScripts/tree/vs_optimization).

To complete the analysis, run all cells in order one-by-one unless otherwise specified, filling in or changing input values as needed. To export a plot, you may want to change the DPI in the `plt.figure` command, i.e. `plt.figure(..., dpi=160)`.

**Requirements**: This notebook requires Python 2, and the following modules:

* `FlowCytometryTools` (install using `pip install flowcytometrytools`)
* `lmfit`
* `ipywidgets`, version 7.2 or later (upgrade if necessary)
* The script `facs_utils.py` must be in the same directory as this notebook.

After installing these modules, be sure to restart the notebook kernel to make sure the modules are available.

*Written by*: Venkatesh Sivaraman, February 2019, adapted from scripts by Dustin Whitney and Theresa Hwang

In [ ]:
### Experiment metadata
# Name:
# Date:
# Info:

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline
import os
from FlowCytometryTools import FCMeasurement
from FlowCytometryTools import ThresholdGate, PolyGate
import pandas as pd 
from lmfit import Model, Parameters, Minimizer
from facs_utils import *
from collections import OrderedDict
from FlowCytometryTools.core.transforms import hlog

In [ ]:
# Dictionary that contains gating information (rerun this cell to reset interactive gate info)
GATE_RESULTS = {}

## Step 0. Preliminaries

Change the values below as appropriate for your experiment.

In [ ]:
# Path to directory containing .fcs files
DATA_PATH = ""

# Number of specimens
NUM_SPECIMENS = 1

# Letter codes used
LETTER_CODES = 'ABCDEFGH'

# Number of samples per letter code
NUM_PER_LETTER = 12

HYPERLOG_B = 100.0

In [ ]:
# Defining axes for various types of gates
scatter_axes = ["FSC-H", "SSC-H"]
fluor_axes = ["Alexa Fluor 680-A", "PE-A"]

# These axes can be hyperlog-transformed if specified
transformable_axes = ["FSC-H", "SSC-H", "SSC-W", "Alexa Fluor 680-A", "PE-A"]

# Expression axis
expression_axis = "PE-A"

In [1]:
# Utility functions to get sample paths and data

def specimen_files(specimen_number, letter=None):
    """Returns the list of fcs files in DATA_PATH corresponding to the given (integer) specimen number. If 
    letter is not None, also looks for specimens with the given letter code."""
    prefix = 'Specimen_' + str(specimen_number).zfill(3)
    
    def matches(name):
        if not name.startswith(prefix): return False
        if letter is not None and ("_" + letter) not in name: return False
        return True
    
    paths = [os.path.join(DATA_PATH, path) for path in os.listdir(DATA_PATH) if matches(path)]
    return sorted(paths, key=lambda path: path[path.find(".fcs") - 3:])

def get_sample_path(specimen_number, letter, sample_number):
    """Gets the path for the sample with the given specimen number, letter code, and sample number for the letter code."""
    prefix = 'Specimen_' + str(specimen_number).zfill(3) + '_' + letter + str(sample_number) + '_'
    
    paths = [os.path.join(DATA_PATH, path) for path in os.listdir(DATA_PATH) if path.startswith(prefix)]
    assert len(paths) == 1, "Multiple paths satisfy sample path condition"
    return paths[0]

def transform_sample(sample):
    """Performs the standard hyperlog transformation on the given sample."""
    return sample.transform('hlog', channels=transformable_axes, b=HYPERLOG_B)
    
def get_sample(path, id_name, transform=False):
    """Gets a measurement from the given fcs file path, transforming its scatter values using a hyperlog 
    transformation if specified."""
    sample = FCMeasurement(ID=id_name, datafile=path)
    if transform:
        return transform_sample(sample)
    return sample

## Step 1. Forward-Scatter and Side-Scatter Gating

In [ ]:
# Get sample files to test the scatter gates on

test_files = specimen_files(1, letter='A')
test_samples = [get_sample(path, "Test", transform=True) for path in test_files]

test_file = test_files[0]
test_sample = test_samples[0]

**NOTE**: The following cell opens the interactive gate drawing tool. If no polygon gate is desired, skip this cell, and in the subsequent cell set `scatter_gate` to `None` before running. Alternatively, if you already know the vertices for the polygon you want, skip this cell and replace `GATE_RESULTS['scatter_gate']` in the first line with the list of coordinates.

In [ ]:
# Choose the number of vertices to use on the gate.
num_points = 3

# Opens the interactive gate drawing tool.
# Use test_samples to plot all 'A' samples together, or test_sample to plot just the first sample (may be faster).
vertex_control(test_samples, scatter_axes, num_points, GATE_RESULTS, 'scatter_gate', log=False)

In [ ]:
# Creates the main scatter gate using the interactive results. Set scatter_gate to None if no polygon gate is desired.
gate_vertices = GATE_RESULTS['scatter_gate']
scatter_gate = PolyGate(gate_vertices, scatter_axes, region='in', name='scatter_gate')
print "Gate created with vertices:", [tuple(row) for row in gate_vertices]

# Create some threshold gates to further filter the data.
scatter_threshold_gates = [
    # ThresholdGate(0, 'FSC-H', region='above'),
    # ThresholdGate(9000, 'SSC-H', region='below'),
    # ThresholdGate(300, 'SSC-H', region='above')
]

def gate_by_scatter(sample):
    """Gates the given FCMeasurement using the scatter_gate and the list of scatter_threshold_gates. 
    Returns the new gated sample."""
    tsample = sample
    if scatter_gate is not None:
        tsample = tsample.gate(scatter_gate)
    for gate in scatter_threshold_gates:
        tsample = tsample.gate(gate)
    return tsample

In [ ]:
# Show several examples with this gate
files = specimen_files(1, letter='A')
gates_to_show = ([scatter_gate] if scatter_gate is not None else []) + scatter_threshold_gates
plt.figure(figsize=(16, 10))

for i, path in enumerate(files[:24]):
    plt.subplot(4, 6, i + 1)
    
    # Get the sample, gate it, and plot it
    sample = get_sample(path, "Sample {}".format(i), transform=True)
    filtered_sample = gate_by_scatter(sample)
    filtered_sample.plot(scatter_axes, ax=plt.gca(), kind='scatter', color='r', s=1, gates=gates_to_show)
    
    plt.title(os.path.splitext(os.path.basename(path))[0])
    
plt.tight_layout()
plt.show()

## Step 2. Fluorescence Gating

In [ ]:
# Gate the test samples from above
fluor_samples = [gate_by_scatter(samp) for samp in test_samples]
fluor_sample = fluor_samples[0]

In [ ]:
# Take a look at the first sample, and all the samples
plt.figure()
fluor_sample.plot(fluor_axes, ax=plt.gca(), kind='scatter', color='r', s=1);
plt.figure()
for sample in fluor_samples:
    sample.plot(fluor_axes, ax=plt.gca(), kind='scatter', color='r', s=1);

**NOTE**: The following cell opens the interactive gate drawing tool. If no polygon gate is desired, skip this cell, and in the subsequent cell set `fluor_gate` to `None` before running. Alternatively, if you already know the vertices for the polygon you want, skip this cell and replace `GATE_RESULTS['fluor_gate']` in the first line with the list of coordinates.

In [ ]:
# Choose the number of vertices to use on the gate.
num_points = 3

# Use fluor_samples to plot all 'A' samples together, or fluor_sample to plot just the first sample (may be faster).
vertex_control(fluor_samples, fluor_axes, num_points, GATE_RESULTS, 'fluor_gate', log=False)

In [ ]:
# Creates the main polygon fluorescence gate using the interactive results. 
# Set fluor_gate to None if no polygon gate is desired.
gate_vertices = GATE_RESULTS['fluor_gate']
fluor_gate = PolyGate(gate_vertices, fluor_axes, region='in', name='fluor_gate')
print "Gate created with vertices:", [tuple(row) for row in gate_vertices]


# Create some threshold gates to further filter the data.
fluor_threshold_gates = [
    # ThresholdGate(1, 'PE-A', region = 'above'),
    # ThresholdGate(4000, 'Alexa Fluor 680-A', region = 'above'),
]

def gate_by_fluorescence(sample):
    """Gates the given FCMeasurement using the fluor_gate and the list of fluor_threshold_gates. 
    Returns the new gated sample."""
    tsample = sample
    if fluor_gate is not None:
        tsample = sample.gate(fluor_gate)
    for gate in fluor_threshold_gates:
        tsample = tsample.gate(gate)
    return tsample

In [ ]:
# Show several examples with all gates applied
files = specimen_files(1, letter='A')
gates_to_show = ([fluor_gate] if fluor_gate is not None else []) + fluor_threshold_gates
plt.figure(figsize=(16, 10))

for i, path in enumerate(files[:24]):
    plt.subplot(4, 6, i + 1)
    
    # Get the sample, gate it, and plot it
    sample = get_sample(path, "Sample {}".format(i), transform=True)
    filtered_sample = gate_by_fluorescence(gate_by_scatter(sample))
    filtered_sample.plot(fluor_axes, ax=plt.gca(), kind='scatter', color='r', s=1, gates=gates_to_show)
    
    plt.title(os.path.splitext(os.path.basename(path))[0])
    
plt.tight_layout()
plt.show()

## Step 3. $K_d$ Estimation

Up until now we have only been working with a small subset of the data - now, we will perform the analysis on all samples. 

**A note on transformations:** All of the gates we have created are in hyperlog-space, so this script is careful to apply any gates in that space. The best practice for fitting, however, is usually to perform the fit on linear (un-transformed) medians. If the `linear_medians` variable below is set to `True`, the median of the un-transformed data will be used (still gated in log space); otherwise, the median of the transformed data will be used.

In [ ]:
# If this is set to True, we will use the untransformed data to get the medians; 
# otherwise, we use the log-transformed data
linear_medians = True

# Get the medians of every gated sample
medians = np.zeros((NUM_SPECIMENS, len(LETTER_CODES), NUM_PER_LETTER))

for specimen in range(1, NUM_SPECIMENS + 1):
    for letter_idx, letter in enumerate(LETTER_CODES):
        for sample_num in range(1, NUM_PER_LETTER + 1):
            # Get the path for this sample
            path = get_sample_path(specimen, letter, sample_num)
            id_name = os.path.splitext(os.path.basename(path))[0]
            
            # Load the sample and gate it
            sample = get_sample(path, id_name, transform=False)
            tsample = transform_sample(sample)
            filtered_tsample = gate_by_fluorescence(gate_by_scatter(tsample))
            filtered_sample_data = sample.get_data().loc[filtered_tsample.get_data().index]
            
            # Compute median of expression axis values
            if linear_medians:
                medians[specimen - 1, letter_idx, sample_num - 1] = filtered_sample_data[expression_axis].median()
            else:
                medians[specimen - 1, letter_idx, sample_num - 1] = filtered_tsample.data[expression_axis].median()

In [ ]:
# Plot the medians
specimen_to_plot = 1
letter_to_plot = 'C'
plt.figure(figsize=(16, 6))

for sample_num in range(1, NUM_PER_LETTER + 1):
    # Get the path for this sample
    path = get_sample_path(specimen_to_plot, letter_to_plot, sample_num)
    id_name = os.path.splitext(os.path.basename(path))[0]
    median = medians[specimen_to_plot - 1, LETTER_CODES.find(letter_to_plot), sample_num - 1]

    # Load the sample and gate it
    sample = get_sample(path, id_name, transform=True)
    filtered_sample = gate_by_fluorescence(gate_by_scatter(sample))
    
    # Plot
    plt.subplot(2, int(ceil(NUM_PER_LETTER / 2.0)), sample_num)
    filtered_sample.plot(fluor_axes, ax=plt.gca(), kind='scatter', color='r', s=1)
    # The median should be hlog-transformed for display if it's on a linear scale
    plt.hlines(hlog(median, b=HYPERLOG_B) if linear_medians else median, *plt.xlim(), color='b')
    plt.title(id_name)
    
plt.tight_layout()
plt.show()

In [ ]:
# Define your concentrations here, in the order that they would appear in the below mapping.
concentrations = np.array([20, 5, 1.25, .312, 0.08, 0.01])

We need a mapping that tells us which letter codes and indexes belong to the same titration. The default behavior, implemented below, is to reshape the matrix to `(NUM_SPECIMENS, x, len(concentrations))`, where `x` is the number of rows needed to use up all 96 values per specimen. For example, with 6 concentrations, the first titration will consist of samples A1 through A6; the second will consist of A7 through A12; the third B1 through B6; and so on. 

If your plate is organized differently, you'll likely need to implement a different behavior to create `median_matrix` such that its first dimension is the number of specimens and its third dimension is equal to the number of concentrations.

In [ ]:
vals_per_specimen = medians.shape[1] * medians.shape[2]
assert vals_per_specimen / len(concentrations) == round(vals_per_specimen / float(len(concentrations))), "Number of concentrations does not divide the total number of samples evenly. Please implement a different mapping method to obtain the median fluorescence matrix."
median_matrix = medians.reshape(NUM_SPECIMENS, -1, len(concentrations))

print "Median matrix: ", median_matrix

In [ ]:
# Give each set of samples a more descriptive label for bookkeeping
labels = ["Sample 1", "Sample 2", "Sample 3", "Sample 4",
         "Sample 5", "Sample 6", "Sample 7", "Sample 8",
         "Sample 9", "Sample 10", "Sample 11", "Sample 12",
         "Sample 13", "Sample 14", "Sample 15", "Sample 16"]

assert len(labels) == median_matrix.shape[0] * median_matrix.shape[1], "Need {} total labels, got {}".format(median_matrix.shape[0] * median_matrix.shape[1], len(labels))

In [ ]:
# Now we compute all the Kds. 
# Initial parameters:
init_lower = 0
init_upper = 4000
init_kd = 1
plot_fits = True

# List of dictionaries of fit results
fit_results = []

label_index = 0
    
for specimen in range(median_matrix.shape[0]):
    
    if plot_fits: plt.figure(figsize=(16, 16))
    for i, dataset in enumerate(median_matrix[specimen]):
        
        if plot_fits:
            plt.subplot(int(ceil(median_matrix.shape[1] / 4.0)), 4, i + 1)
        
        # Compute fit and save to results dictionary
        kd, sat, init, err, chisqr, r2 = run_lmfit(concentrations, dataset, init_lower, init_upper, init_kd, plot_fits)
        result = [("label", labels[label_index]), 
                  ("kd", kd), ("sat", sat), ("init", init),
                  ("err", err), ("chisqr", chisqr), ("r2", r2)]
        for conc, val in zip(concentrations, dataset):
            result.append(("conc_{}".format(conc), val))
        fit_results.append(OrderedDict(result))
        label_index += 1
        
        if plot_fits:
            plt.title("Titration {} (Kd = {:.3g})".format(i + 1, kd))
            
    if plot_fits:
        plt.tight_layout()
        plt.show()

In [ ]:
# Build a dataframe from the results list
results_df = pd.DataFrame(fit_results)
results_df

In [ ]:
# Choose an output path, and write out to CSV
out_path = "kd_list.csv"
results_df.to_csv(out_path, index=False)

That's all, folks!